In [ ]:
from moku.instruments import MultiInstrument, DigitalFilterBox, FrequencyResponseAnalyzer
import control as ct
import altair as alt
import pandas as pd
import numpy as np

## The following function takes a continuous time transfer function in zpk form and returns coefficents for a Moku IIR filter

In [ ]:
from scipy.signal import bilinear_zpk, zpk2sos

def zpk2mokuIIR(az,ap,ak,fs = 39.063e6):
    samplerates = [305.28e3, 4.8828e6, 39.063e6]
    if samplerates.count(fs) == 0:
        raise Exception("{} is not a valid sample rate for a Moku Pro IIR filter".format(fs))

    dz,dp,dk = bilinear_zpk(az,ap,ak,fs)

    sos = zpk2sos(dz,dp,dk)

    if len(sos) > 4:
        raise Exception("{} second-order stages is more than the Moku's maximum of 4".format(len(sos)))

    mokusos = []

    for section in sos:
        mokusos.append([1/section[3],section[0],section[1],section[2],section[4]/section[3],section[5]/section[3]])

    return mokusos


In [ ]:
sample_freq = 39.06e6

In [ ]:
#See the moku user manual for 
def second_order_section(s,b0,b1,b2,a1,a2):
    return ct.tf([s*b0,s*b1,s*b2],[1,a1,a2],1/(sample_freq))

## Here is the transfer function used for the rest of the script

In [ ]:
z = [-500,-35000,-50000+40j , -50000-40j]
p = [-1000,-12000,-80000+40j , -80000-40j]
k = 1

ctimemodel = ct.zpk(z,p,k)

## Create the discrete time model

In [ ]:
filter_coefficients = zpk2mokuIIR(z,p,k)

dtimemodel = ct.tf([1],[1],1/(sample_freq))

for coeff in filter_coefficients:
    stage = second_order_section(*coeff)
    dtimemodel = ct.series(dtimemodel,stage)

## Configure the Moku to set and measure an IIR filter

In [ ]:

MIM = MultiInstrument('192.168.50.97', force_connect=True, platform_id=4) #192.168.50.97 is the IP for Moku Pro 1

dfb = MIM.set_instrument(1, DigitalFilterBox)
fra = MIM.set_instrument(2, FrequencyResponseAnalyzer)

connections = [dict(source="Slot1OutA", destination="Slot2InA"),
               dict(source="Slot2OutA", destination="Slot1InA")]

MIM.set_connections(connections=connections)

In [ ]:
# Configure IIR filter 1 of the DFB

dfb.set_custom_filter(1, "39.06MHz", coefficients=filter_coefficients) #note that this should match sample_freq

dfb.set_input_gain(1,gain = 0)
dfb.set_output_gain(1,gain = 0)

dfb.enable_output(1,True,True)

In [ ]:
#configure the frequency sweep
fra.set_sweep(start_frequency=10, stop_frequency=20e6, num_points=256,
                averaging_time=1e-3, averaging_cycles=1, settling_cycles=1,
                settling_time=1e-3)
fra.set_output(1, 0.01)

In [ ]:
delay = fra.start_sweep() 
print(delay)
data = fra.get_data(wait_complete = True)

In [ ]:
MIM.relinquish_ownership()

In [ ]:
df_measured = pd.DataFrame(data = data['ch1'])

df_measured['label'] = 'measured'

df_measured['magnitude'] = 10**(df_measured['magnitude']/20) #convert to magnitude

In [ ]:
omega = df_measured['frequency'].to_numpy()*2*np.pi #convert the sample frequencies to rad/sec

In [ ]:
mag,phase,omega = ct.bode_plot(dtimemodel,omega=omega)
df_dmodel = pd.DataFrame().reindex(columns=df_measured.columns) #create another dataframe with the same columns
df_dmodel['frequency'] = df_measured['frequency']
df_dmodel['magnitude'] = mag
df_dmodel['phase'] = (360*phase/(2*np.pi)+180)%360 -180 #convert to degrees and wrap phase
df_dmodel['label'] = 'discrete time model'

In [ ]:
mag,phase,omega = ct.bode_plot(ctimemodel,omega=omega)
df_cmodel = pd.DataFrame().reindex(columns=df_measured.columns) #create another dataframe with the same columns
df_cmodel['frequency'] = df_measured['frequency']
df_cmodel['magnitude'] = mag
df_cmodel['phase'] = (360*phase/(2*np.pi)+180)%360 -180 #convert to degrees and wrap phase
df_cmodel['label'] = 'continuous time model'

In [ ]:
df = pd.concat([df_dmodel,df_cmodel,df_measured])

In [ ]:
magnitudechart = alt.Chart(df).mark_line(clip=True).encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('magnitude:Q').scale(type="log",domain=(0.5,1.1)),
    color='label:N',
).properties(
    width=400,
    height=200
)

phasechart = alt.Chart(df).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('phase:Q'),
    color='label:N',
).properties(
    width=400,
    height=100
)

alt.vconcat(magnitudechart,phasechart)#.transform_filter(alt.FieldRangePredicate(field='freq', range=[8e6, 1e8]))

In [ ]:
data = df.loc[df['label'] == 'measured']
freq_diff = data['frequency'].diff()*2*np.pi #in rad/s
phase_diff = data['phase'].diff()*2*np.pi/360 #in radians
delay = pd.DataFrame(phase_diff/freq_diff,columns=['delay'])
data = pd.concat([data,delay],axis=1)
data


In [ ]:
delaychart = alt.Chart(data).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log"),
    y=alt.Y('delay:Q'),
    color='label:N',
).properties(
    width=400,
    height=200
)
delaychart

In [ ]:
data['delay'].median()*1e9 #in nanoseconds